# Setup and imports

In [3]:
import re
from pprint import pprint
import json
from langchain_ollama import OllamaLLM, OllamaEmbeddings
import os # remove from here

from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

llama32 = OllamaLLM(model="llama3.2:3b", temperature=0)
embedding = OllamaEmbeddings(model="nomic-embed-text:v1.5")

c:\Users\tiago\anaconda3\envs\Graph_implementation\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

AZURE_DEPLOYMENT_GPT41 = os.getenv("AZURE_DEPLOYMENT_GPT41")
AZURE_DEPLOYMENT_GPT41_NANO = os.getenv("AZURE_DEPLOYMENT_GPT41_NANO")

gpt41_nano = AzureChatOpenAI(
    azure_deployment=AZURE_DEPLOYMENT_GPT41_NANO,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0.0
)

# Data Structures

## Database Schema

In [5]:
def parse_mysql_ddl_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        ddl = f.read()

    # Remove comments and MySQL directives
    ddl = re.sub(r'/\*.*?\*/', '', ddl, flags=re.DOTALL)
    ddl = re.sub(r'--.*?$', '', ddl, flags=re.MULTILINE)
    ddl = re.sub(r'/\!.*?\*/;', '', ddl, flags=re.DOTALL)
    ddl = re.sub(r'/\!.*?\*/', '', ddl, flags=re.DOTALL)

    # Find all CREATE TABLE statements (handles backticks and multiline)
    table_regex = re.compile(
        r'CREATE TABLE\s+`?(\w+)`?\s*\((.*?)\)\s*ENGINE=.*?;',
        re.DOTALL | re.IGNORECASE
    )
    tables = table_regex.findall(ddl)
    result = {}

    for table_name, table_body in tables:
        # Raw DDL - not used anymore
        raw_ddl = f"CREATE TABLE `{table_name}` ({table_body});"

        # Split lines, remove empty and trailing commas
        lines = [line.strip().rstrip(',') for line in table_body.splitlines() if line.strip()]
        columns = []
        primary_keys = []
        foreign_keys = []

        for line in lines:
            # Column definition (starts with backtick or word, not constraint)
            if re.match(r'^`?\w+`?\s', line) and not line.upper().startswith(('PRIMARY KEY', 'FOREIGN KEY', 'CONSTRAINT', 'UNIQUE', 'KEY')):
                col_name = re.match(r'^`?(\w+)`?', line).group(1)
                columns.append(col_name)
            # Primary key
            elif line.upper().startswith('PRIMARY KEY'):
                pk_match = re.search(r'\((.*?)\)', line)
                if pk_match:
                    pk_cols = [col.strip(' `') for col in pk_match.group(1).split(',')]
                    primary_keys.extend(pk_cols)
            # Foreign key
            elif line.upper().startswith('CONSTRAINT') and 'FOREIGN KEY' in line.upper():
                fk_match = re.search(r'FOREIGN KEY\s*\((.*?)\)\s*REFERENCES\s*`?(\w+)`?\s*\((.*?)\)', line, re.IGNORECASE)
                if fk_match:
                    fk_cols = [col.strip(' `') for col in fk_match.group(1).split(',')]
                    ref_table = fk_match.group(2)
                    ref_cols = [col.strip(' `') for col in fk_match.group(3).split(',')]
                    foreign_keys.append({
                        'columns': fk_cols,
                        'ref_table': ref_table,
                        'ref_columns': ref_cols
                    })

        result[table_name] = {
            'columns': columns,
            'primary_keys': primary_keys,
            'foreign_keys': foreign_keys
        }

    return result

In [6]:
schema = parse_mysql_ddl_file('../../database/schema/usable_schema.sql')
pprint(schema)

{'granter_application': {'columns': ['id',
                                     'created_at',
                                     'updated_at',
                                     'state',
                                     'codename',
                                     'title',
                                     'other_data',
                                     'company_id',
                                     'opportunity_id',
                                     'profile_creator_id',
                                     'pricing_option',
                                     'ai_review_state',
                                     'consortium_id',
                                     'sale_confirmed',
                                     'approved_grant_amount',
                                     'success_fee',
                                     'success_payment_amount',
                                     'upfront_payment_amount',
                                     '

## Descriptions of Database tables

In [7]:
with open('../../database/README.md', 'r', encoding='utf-8') as f:
    readme_database_content = f.read()

print(readme_database_content)

# Granter Database Schema
This document explains the relationships between the main tables used in **Granter**, the AI-powered grant application platform.
---
## Tables and Relationships
### **granter_company**
- Represents a company that applies to funding opportunities.
- **Relations:**
  - Can create **granter_applications** for opportunities.
  - Has associated **granter_companyfiles** for company-related context.
  - Has associated **granter_companymemories** (vectorized text snippets for RAG).
---
### **granter_application**
- An application generated for a specific **company** and a specific **opportunity**.
- **Relations:**
  - Belongs to a **granter_company**.
  - Belongs to a **granter_opportunity**.
  - Has associated **granter_applicationfiles** (files relevant to this application).
---
### **granter_applicationfile**
- Files tied to a specific **application**.
- Usually provide company or project-specific context for that application.
- **Relations:**
  - Belongs to a **gr

## External Ontologies

In [8]:
with open("external_ontologies/DINGO-Manchester.omn", "r", encoding="utf-8") as f:
    external_ontology = f.read()
print(external_ontology)

Prefix: owl: <http://www.w3.org/2002/07/owl#>
Prefix: rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
Prefix: rdfs: <http://www.w3.org/2000/01/rdf-schema#>
Prefix: xml: <http://www.w3.org/XML/1998/namespace>
Prefix: xsd: <http://www.w3.org/2001/XMLSchema#>



Ontology: <https://w3id.org/dingo#>


Annotations: 
    <http://purl.org/dc/elements/1.1/contributor> _:0,
    <http://purl.org/dc/elements/1.1/contributor> _:1,
    <http://purl.org/dc/elements/1.1/contributor> _:2,
    <http://purl.org/dc/elements/1.1/contributor> _:4,
    <http://purl.org/dc/elements/1.1/contributor> _:5,
    <http://purl.org/dc/elements/1.1/issued> "2018-10-26T00:00:00Z",
    <http://purl.org/dc/elements/1.1/title> "DINGO Ontology",
    <http://purl.org/dc/terms/creator> _:3,
    <http://purl.org/dc/terms/modified> "2020-01-10T10:20:10Z",
    rdfs:comment "The DINGO ontology (Data Integration for Grant Ontology) defines the terms of the DINGO vocabulary and provides a machine readable extensible framework t

## Lexical Views

### Lex(S)

In [9]:
def extract_schema_lexical_view(schema_dict) -> list:
    """
    does not repeat column names that appear more than once
    """
    lexical_view = []
    
    # Process tables in the order they appear in the dictionary
    for table_name in schema_dict.keys():
        table_data = schema_dict[table_name]
        
        # Add table name first
        lexical_view.append(table_name)
        
        # Add columns in the order they appear in the list
        for column in table_data['columns']:
            lexical_view.append(column)
        
        # Add primary keys in order (if any)
        for pk in table_data['primary_keys']:
            lexical_view.append(pk)
        
        # Add foreign key information in order (if any)
        for fk in table_data['foreign_keys']:
            lexical_view.append(fk['ref_table'])  # Referenced table name
            for ref_col in fk['ref_columns']:
                lexical_view.append(ref_col)
                
    # Remove duplicates while preserving order
    seen = set()
    unique_lexical_view = []
    for item in lexical_view:
        if item not in seen:
            seen.add(item)
            unique_lexical_view.append(item)
    return unique_lexical_view

In [10]:
def extract_schema_lexical_view(schema_dict) -> list:
    """
    Returns format: ['tablename: column1, column2, column3', 'tablename2: column1, column2']
    """
    lexical_view = []
    
    # Process tables in the order they appear in the dictionary
    for table_name in schema_dict.keys():
        table_data = schema_dict[table_name]
        
        # Get all columns for this table
        columns = table_data['columns']
        
        # Create the formatted string for this table
        if columns:
            columns_str = ', '.join(columns)
            lexical_view.append(f"{table_name}: {columns_str}")
        else:
            lexical_view.append(f"{table_name}: ")
    
    return lexical_view

In [11]:
extract_schema_lexical_view(schema)

['granter_application: id, created_at, updated_at, state, codename, title, other_data, company_id, opportunity_id, profile_creator_id, pricing_option, ai_review_state, consortium_id, sale_confirmed, approved_grant_amount, success_fee, success_payment_amount, upfront_payment_amount, initial_summary, writer_mode',
 'granter_applicationfile: id, created_at, updated_at, name, uploaded_file, vector_indexed, type, application_id, document_type_id',
 'granter_company: id, created_at, updated_at, name, legal_name, address, post_code, city, country, url, legal_form, activity_date, company_id, tax_id, social_security_id, is_sme, employees_n, annual_revenue, description, cover_image, website_data, owner_id, stripe_customer_id, digital_component, future_goals_infrastructure, future_goals_internationalization, future_goals_machinery_and_equipments, future_goals_marketing, future_goals_rd, green_component, team_description, future_goals_hr, onboarding_state, matches_generating, userless, cae, whitel

### Lex(E)

In [12]:
def extract_ontology_lexical_view(ontology_path: str) -> list:
    """
    Returns format: ['label: comment', 'label2: comment2']
    Pairs labels with their corresponding comments
    """
    with open(ontology_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    lexical_view = []
    
    # Find all rdfs:label and rdfs:comment matches with their positions
    label_pattern = r'rdfs:label\s+"([^"]+)"'
    comment_pattern = r'rdfs:comment\s+"([^"]+)"'
    
    # Get all matches in order
    labels = [match.group(1).replace(" ", "") for match in re.finditer(label_pattern, content)]
    comments = [match.group(1) for match in re.finditer(comment_pattern, content)]
    comments.pop(0) # Remove the first comment as it does not correspond to any label

    # Pair them sequentially (comment[i] goes with label[i])
    for i in range(len(labels)):
        if i < len(comments):
            lexical_view.append(f"{labels[i]}: {comments[i]}")
        else:
            lexical_view.append(f"{labels[i]}: ")
    
    return lexical_view

In [13]:
extract_ontology_lexical_view('external_ontologies/DINGO-Manchester.omn')

['academic_degree: Qualification awarded to the person taking the role by a higher education organisation, normally at a college or university.',
 'affiliation: An organization that this person is affiliated with. For example, a school/university, a club, or a team.',
 'agency_identifier: Identifier (for example of Project, Grant, GrantPayment, PersonRole instance, OrganizationRole instance, FundingScheme, Person, Organisation) in the database/archive of the funding agency/funding body/funder.',
 'baseline_academic_degree: Academic degree used as minimal/reference point for establishing a set of age limits or other classification.',
 'beneficiary: The relation between the Grant (subject) and a (one of the) person(s) or organisation(s) receiving benefits from it identified by his/her/its role.',
 'beneficiary_of: The relation between a (one of the) person(s) or organisation(s) receiving benefits from a Grant identified by his/her/its role (subject), and the Grant.',
 'budget: Amount of 

# RAG of Relevant Knowledge

The **retrieval** will be done over:<br>
+ The Growing core ontology<br>
+ The External Ontology Repository<br>
+ The Textual Descriptions of the RDB

## Chunking (the textual descriptions of the database)

In [14]:
def chunk_text(text: str, chunk_size=1200, chunk_overlap=100):
    """
    Chunk text and return plain strings (no metadata)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

In [15]:
readme_database_chunks = chunk_text(readme_database_content, 500)
print(readme_database_chunks[0])

# Granter Database Schema
This document explains the relationships between the main tables used in **Granter**, the AI-powered grant application platform.
---
## Tables and Relationships
### **granter_company**
- Represents a company that applies to funding opportunities.
- **Relations:**
  - Can create **granter_applications** for opportunities.
  - Has associated **granter_companyfiles** for company-related context.


## Embedding amd Indexing

In [16]:
INDEX_PATH_CORE = "../../storage/faiss_RIGOR/core"
INDEX_PATH_EXTERNAL_ONT = "../../storage/faiss_RIGOR/external_ontologies"
INDEX_PATH_TEXTUAL_DESC = "../../storage/faiss_RIGOR/textual_descriptions"

def embed_and_index(text_chunks: list, index_path: str):
    """
    Embed and index text chunks in the given directory
    """
    os.makedirs(os.path.dirname(index_path), exist_ok=True)

    vectorstore = FAISS.from_texts(text_chunks, embedding)
    vectorstore.save_local(index_path)
    print(f"Index saved at {index_path}. {len(text_chunks)} text chunks embedded and stored.")

In [17]:
#lexS = extract_schema_lexical_view(schema)
lexE = extract_ontology_lexical_view('external_ontologies/DINGO-Manchester.omn')

index_list = [INDEX_PATH_EXTERNAL_ONT, INDEX_PATH_TEXTUAL_DESC]
content_list = [lexE, readme_database_chunks]

for i, index_path in enumerate(content_list):
    embed_and_index(index_path, index_list[i])
    print(f"Indexed {len(index_path)} text chunks in {index_list[i]}.")


Index saved at ../../storage/faiss_RIGOR/external_ontologies. 108 text chunks embedded and stored.
Indexed 108 text chunks in ../../storage/faiss_RIGOR/external_ontologies.
Index saved at ../../storage/faiss_RIGOR/textual_descriptions. 12 text chunks embedded and stored.
Indexed 12 text chunks in ../../storage/faiss_RIGOR/textual_descriptions.


## Retrieval Functions

In [18]:
def retrieve(input, index_path, k=4):
    vectorstore = FAISS.load_local(index_path, embedding, 
                                   allow_dangerous_deserialization=True)
    docs = vectorstore.similarity_search(input, k=k)

    return [doc.page_content for doc in docs]

#print(retrieve("application file", INDEX_PATH_TEXTUAL_DESC, 2)[0])
print(retrieve("application", INDEX_PATH_EXTERNAL_ONT, 2)[0])

organisation_location: Country or geographical administrative entity where an organisation must be established to apply for funding.


In [19]:
def docTable(relation: str):
    """
    returns the textual description of a relation r
    """
    return retrieve(f"relation: {relation}", INDEX_PATH_TEXTUAL_DESC, 2)


def docAttr(relation: str, attribute: str):
    """
    returns the textual description of an attribute a in relation r
    """
    return retrieve(f"relation: {relation}, attribute: {attribute}", INDEX_PATH_TEXTUAL_DESC, 1)

# LLM Prompt Construction and Ontology Generation

In [20]:
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

groqllm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY,
    temperature=0
)

In [21]:
table = schema['granter_application']
core_ontology = ""

prompt = f"""
    Generate ontology elements with provenance annotations for database table granter_application based on:

    [CONTEXT]
    - Database Schema of the table \n{json.dumps(table)}\n
    - Take semantics from the External Ontology Knowledge \n{"\n".join(retrieve('granter_application', INDEX_PATH_EXTERNAL_ONT))}\n
    - Take semantics from the Relevant Documents \n{"\n".join(docTable('granter_application'))}

    [INSTRUCTIONS]
    1. Include these elements:
        Classes (subclass of Thing)
        Data properties with domain/range
        Object properties with domain/range
        Use only one rdfs:domain and one rdfs:range per property. If multiple options exist, select the most general or create a shared superclass.
    3. Do not create a property named "is". Use rdf:type for instance membership, rdfs:subClassOf for class hierarchies, and owl:sameAs for instance equality.
    4. Use this format example:

    Prefix: prov: <http://www.w3.org/ns/prov#>
    Prefix: xsd: <http://www.w3.org/2001/XMLSchema#>
    Ontology: <http://example.org/my-ontology>

    Class: granter_application
    Annotations:
        prov:wasDerivedFrom <http://example.org/provenance/granter_application>

    DataProperty: has_column_name
    Domain: granter_application
    Range: xsd:string
    Annotations:
        prov:wasDerivedFrom <http://example.org/provenance/granter_application/column_name>

    ObjectProperty: relates_to_table
    Domain: granter_application
    Range: RelatedTable
    Annotations:
        prov:wasDerivedFrom <http://example.org/provenance/granter_application/fk_column>


    Only output Manchester Syntax and nothing else. [OUTPUT]
"""

In [22]:
print("\n".join(retrieve('granter_application', INDEX_PATH_EXTERNAL_ONT)))

recipient: The receiver of a GrantPayment or receiver/beneficiary of a GrantShare.
GrantPayment: The class for grant payments: a single payment to a recipient or beneficiary within a Grant.
Grant: The class for grant: a disbursed fund payed to a recipient or beneficiary and the process for it.
CareerPurposeCriterion: Specifies that grants are awarded on the basis of career-related goals and applicants must be at a defined career stage (defined academic degree) to be achieved.


In [23]:
print("\n".join(docTable('granter_application')))

---
### **granter_companymemory**
- Small text snippets stored as vectors for RAG purposes.
- Help provide semantic context when drafting applications.
- **Relations:**
  - Belongs to a **granter_company**.
---
### **granter_opportunity**
- Represents a funding opportunity (grant, tax incentive, investment, or loan).
- **Relations:**
  - Can have multiple **granter_applications** from different companies.
  - Has associated **granter_opportunityfiles**.
- Has associated **granter_companyfiles** for company-related context.
  - Has associated **granter_companymemories** (vectorized text snippets for RAG).
---
### **granter_application**
- An application generated for a specific **company** and a specific **opportunity**.
- **Relations:**
  - Belongs to a **granter_company**.
  - Belongs to a **granter_opportunity**.
  - Has associated **granter_applicationfiles** (files relevant to this application).
---
### **granter_applicationfile**


In [24]:
delta_ontology = groqllm.invoke(prompt)

In [25]:
print(delta_ontology.content)

Prefix: prov: <http://www.w3.org/ns/prov#>
Prefix: xsd: <http://www.w3.org/2001/XMLSchema#>
Ontology: <http://example.org/granter_ontology>

Class: granter_application
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_application>.

Class: granter_company
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_company>.

Class: granter_opportunity
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_opportunity>.

Class: granter_profile
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_profile>.

Class: granter_consortium
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_consortium>.

Class: Grant
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/Grant>.

Class: GrantPayment
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/GrantPayment>.

Class: GrantShare
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/GrantSh

In [26]:
type(delta_ontology.content)

str

# Validation and Refinement 

### Prompt

In [27]:
evaluator_prompt = f"""
You are an expert in OWL 2 DL ontology modeling and validation.

Your task is to review the following delta ontology fragment generated from a relational database table, along with its schema and relevant context.

[DELTA-ONTOLOGY]
{delta_ontology}

[DATABASE SCHEMA]
{table}

[CORE ONTOLOGY CONTEXT]
(empty as of now)

[VALIDATION CRITERIA]
1. **Coherence with Core Ontology**  
   - Do NOT redefine an existing class, property, or concept already present in the core ontology with the same meaning.
   - Reuse existing ontology elements where possible instead of creating duplicates.

2. **Alignment with Input Table Schema**  
   - Every significant column and foreign key in the table must be represented as an appropriate ontology element (class, data property, or object property).
   - Naming should reflect the database semantics clearly and consistently.

3. **Syntactic Validity**  
   - The ontology must conform to the OWL 2 DL profile and valid Manchester Syntax.
   - Only one `rdfs:domain` and one `rdfs:range` per property.

4. **Logical Consistency**  
   - No contradictory class axioms or property constraints.
   - No circular subclass relationships.
   - Correct choice between object properties and data properties.

5. **Clarity and Naming Quality**  
   - Use self-explanatory, domain-relevant names.
   - Avoid generic or meaningless labels (e.g., "Entity1", "PropertyA").
   - All properties should follow consistent naming patterns (e.g., `has_`, `is_...Of`).

[YOUR TASK]
- Check the delta ontology fragment against all criteria above.
- If issues are found, provide a corrected version of the ontology in valid Manchester Syntax.
- Make minimal necessary changes to preserve the author's intent while ensuring correctness and OWL 2 DL compliance.
- Ensure all elements keep their provenance annotations.

[OUTPUT FORMAT]
Respond ONLY with:
1. "Status: PASS" if the ontology fragment meets all criteria, or "Status: FAIL" if it does not.
2. If FAIL, provide:
   a. A short bullet list of the issues found.
   b. A corrected Manchester Syntax version of the ontology fragment, and enclose it between triple backticks (```), on their own lines.

Do NOT include any other commentary outside this format.
Always enclose the corrected ontology with triple backticks for programmatic extraction.
"""

In [28]:
revision = groqllm.invoke(evaluator_prompt)
print(revision.content)

Status: FAIL

* Issue 1: The ontology contains duplicate properties with the same meaning.
* Issue 2: The ontology contains circular subclass relationships.
* Issue 3: The ontology contains incorrect choice between object properties and data properties.
* Issue 4: The ontology contains contradictory class axioms or property constraints.

````
Prefix: prov: <http://www.w3.org/ns/prov#>
Prefix: xsd: <http://www.w3.org/2001/XMLSchema#>

Ontology: <http://example.org/granter_ontology>

Class: granter_application
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_application>.

Class: granter_company
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_company>.

Class: granter_opportunity
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_opportunity>.

Class: granter_profile
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_profile>.

Class: granter_consortium
Annotations:
    prov:wasDerive

In [29]:
# Check if validation passed
if "Status: PASS" in revision.content:
    print(delta_ontology.content, True)
else:
    # Extract corrected ontology from revision
    lines = revision.content.split('\n')
    corrected_start = False
    corrected_ontology = []
    
    for line in lines:
        if corrected_start:
            corrected_ontology.append(line)
        elif line.strip().startswith('Class:') or line.strip().startswith('DataProperty:') or line.strip().startswith('ObjectProperty:'):
            corrected_start = True
            corrected_ontology.append(line)
    
    if corrected_ontology:
        print('\n'.join(corrected_ontology), True)
    else:
        print(f"Warning: Could not extract corrected ontology for table {'granter_application'}")
        print(delta_ontology, False)

Class: granter_application
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_application>.

Class: granter_company
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_company>.

Class: granter_opportunity
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_opportunity>.

Class: granter_profile
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_profile>.

Class: granter_consortium
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/granter_consortium>.

Class: Grant
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/Grant>.

Class: GrantPayment
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/GrantPayment>.

Class: GrantShare
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/GrantShare>.

Class: CareerPurposeCriterion
Annotations:
    prov:wasDerivedFrom <http://example.org/provenance/CareerPurposeCriterion>.

Class: Rel

# Iterative Integration and Completion

Recreate and Embed Lex(Ot) at each iteration to then retrieve it

## Function to extract corrected ontology from Judge LLM output

In [30]:
def extract_ontology_block(text: str) -> str:
    """
    Extracts the ontology code block (Manchester Syntax) from a mixed LLM output.
    Looks for content between triple backticks (```).

    Parameters:
        text (str): The full output from the LLM.

    Returns:
        str: The extracted ontology block as a string, or an empty string if not found.
    """
    inside_block = False
    lines = []
    
    for line in text.splitlines():
        if line.strip().startswith("```"):
            if inside_block:
                break  # end of block
            else:
                inside_block = True
                continue  # skip the line with ```
        if inside_block:
            lines.append(line)
    
    return "\n".join(lines).strip()

## Function to create delta ontology

In [31]:
from rdflib import Graph

def validate_turtle_string(turtle_str: str) -> tuple[bool, str]:
    """
    Validates whether the input string is valid Turtle syntax.

    Returns:
        - is_valid (bool): True if valid, False otherwise
        - error_msg (str): Error message if invalid, else empty
    """
    try:
        g = Graph()
        g.parse(data=turtle_str, format="turtle")
        return True, ""
    except Exception as e:
        return False, str(e)

In [ ]:
def build_table_ontology(table_name: str, table_schema: dict, core_ontology: str):
    prompt = f"""
    Generate ontology elements with provenance annotations for database table {table_name}, based on:

    [CONTEXT]
    - Database Schema of the table \n{json.dumps(table_schema)}\n
    - Take semantics from the External Ontology Knowledge \n{"\n".join(retrieve(table_name, INDEX_PATH_EXTERNAL_ONT))}\n
    - Take semantics from the Relevant Documents \n{"\n".join(docTable(table_name))}

    [INSTRUCTIONS]
    1. Include these elements:
        Classes (subclass of owl:Thing)
        Data properties with domain/range
        Object properties with domain/range
        Use only one Domain: and one Range: per property. If multiple options exist, select the most general or create a shared superclass.
    2. Use Turtle/RDF syntax format
    3. Do not create a property named "is". Use rdf:type for instance membership, rdfs:subClassOf for class hierarchies, and owl:sameAs for instance equality.
    4. Use this format example:

    @prefix prov: <http://www.w3.org/ns/prov#> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    @prefix owl: <http://www.w3.org/2002/07/owl#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix : <http://example.org/my-ontology#> .

    :{table_name} a owl:Class ;
        prov:wasDerivedFrom <http://example.org/provenance/{table_name}> .

    :has_column_name a owl:DatatypeProperty ;
        rdfs:domain :{table_name} ;
        rdfs:range xsd:string ;
        prov:wasDerivedFrom <http://example.org/provenance/{table_name}/column_name> .

    :relates_to_table a owl:ObjectProperty ;
        rdfs:domain :{table_name} ;
        rdfs:range :RelatedTable ;
        prov:wasDerivedFrom <http://example.org/provenance/{table_name}/fk_column> .

    Only output Turtle syntax and nothing else. [OUTPUT]
    """

    delta_ontology = gpt41_nano.invoke(prompt).content




    is_valid, error_message = validate_turtle_string(delta_ontology)

    if not is_valid:
        additional_syntax_hint = f"""
    [SYNTAX ISSUES DETECTED]

    Note: The delta ontology provided is NOT valid Turtle syntax.

    Please make the following minimal syntax corrections before further validation:
    - Declare all prefixes used in the ontology explicitly (e.g., rdf:, rdfs:, owl:, xsd:, prov:).
    - Use angle brackets <...> for all full IRI references.
    - Each property must have exactly one rdfs:domain and one rdfs:range.
    - Ensure rdfs:range only uses valid OWL classes or XSD types (e.g., xsd:string).
    - Remove annotation-style keywords like 'Annotations:' and instead use Turtle triples for metadata.
    - Avoid misplaced punctuation or unclosed blocks.

    Parsing error: {error_message}
    """
    else:
        additional_syntax_hint = ""




    evaluator_prompt = f"""
    You are an expert in OWL 2 DL ontology modeling and validation.

    Your task is to review the following delta ontology fragment generated from a relational database table, along with its schema and relevant context.

    [DELTA-ONTOLOGY]
    {delta_ontology}

    [DATABASE SCHEMA]
    {table_schema}

    [CORE ONTOLOGY CONTEXT]
    {core_ontology if core_ontology else "(empty as of now)"}

    {additional_syntax_hint}

    [VALIDATION CRITERIA]
    1. **Coherence with Core Ontology**  
    - Do NOT redefine an existing class, property, or concept already present in the core ontology with the same meaning.
    - Reuse existing ontology elements where possible instead of creating duplicates.

    2. **Alignment with Input Table Schema**  
    - Every significant column and foreign key in the table must be represented as an appropriate ontology element (class, data property, or object property).
    - Naming should reflect the database semantics clearly and consistently.

    3. **Syntactic Validity**  
    - The ontology must conform to valid Turtle/RDF syntax.
    - Use proper prefixes and IRI declarations.
    - Exactly one rdfs:domain and one rdfs:range per property.

    4. **Logical Consistency**  
    - No contradictory class axioms or property constraints.
    - No circular subclass relationships.
    - Correct choice between object properties and data properties.

    5. **Clarity and Naming Quality**  
    - Use self-explanatory, domain-relevant names.
    - Avoid generic or meaningless labels.
    - All properties should follow consistent naming patterns.

    [YOUR TASK]
    - Check the delta ontology fragment against all criteria above.
    - If issues are found, provide a corrected version of the ontology in valid Turtle syntax.
    - Make minimal necessary changes to preserve the author's intent while ensuring correctness and OWL 2 DL compliance.
    - Ensure all elements keep their provenance annotations.

    [OUTPUT FORMAT]
    Respond ONLY with:
    1. "Status: PASS" if the ontology fragment meets all criteria, or "Status: FAIL" if it does not.
    2. If FAIL, provide:
        a. A short bullet list of the issues found.
        b. A corrected Turtle syntax version of the ontology fragment, and enclose it between triple backticks (```), on their own lines.

    Do NOT include any other commentary outside this format.
    Always enclose the corrected ontology with triple backticks for programmatic extraction.
    """

    revision = gpt41_nano.invoke(evaluator_prompt).content

    
    if "Status: PASS" in revision:
        print("JudgeLLM Validation succeeded.")
        valid = validate_turtle_string(delta_ontology)
        print(valid)
        if valid[0]:
            return delta_ontology
        else:
            print("Syntax Validation failed.\n\n")
            return delta_ontology
    else:
        print("JudgeLLM Validation failed. Corrected Version:")
        valid = validate_turtle_string(extract_ontology_block(revision))
        print(valid)
        if valid[0]:
            return extract_ontology_block(revision)
        else:
            print("Syntax Validation failed.\n\n")
            return extract_ontology_block(revision)

In [33]:
#core = build_table_ontology('granter_application', schema['granter_application'], '')
#print(core)

In [34]:
#validate_turtle_string(core)

In [35]:
#delta = build_table_ontology('granter_company', schema['granter_company'], '')
#print(delta)

In [36]:
#validate_turtle_string(delta)

## Merging the ontologies

In [37]:
from rdflib import Graph

def merge_ontologies_ttl(core_ttl: str, delta_ttl: str) -> str:
    """
    Merge two OWL ontologies in Turtle format (as strings) and return the merged ontology as a Turtle string.
    Uses RDFLib's built-in graph merging with automatic triple deduplication.
    
    Parameters:
        core_ttl (str): Core ontology (Turtle format string)
        delta_ttl (str): Delta ontology (Turtle format string)
        
    Returns:
        str: Merged ontology in Turtle syntax
    """
    # Parse both ontologies
    core_graph = Graph()
    core_graph.parse(data=core_ttl, format="turtle")

    delta_graph = Graph()
    delta_graph.parse(data=delta_ttl, format="turtle")

    # Merge delta into core
    core_graph += delta_graph  # RDFLib automatically deduplicates identical triples

    # Serialize the merged ontology back to Turtle
    return core_graph.serialize(format="turtle")


In [38]:
#Ot1 = merge_ontologies_ttl(core, delta)
#print(Ot1)

In [39]:
#type(Ot1)

In [40]:
#validate_turtle_string(Ot1)

# Running the whole thing

In [44]:
table_names = list(schema.keys())
no_fk_tables = [name for name in table_names if not schema[name]['foreign_keys']]
fk_tables = [name for name in table_names if schema[name]['foreign_keys']]
processing_order = no_fk_tables + fk_tables

print(f"Processing {len(processing_order)} tables: {processing_order}")

core_ontology = ""
successful = []
failed = []

for table in processing_order:
    print(f"Processing {table}...", end=" ")
    
    try:
        delta = build_table_ontology(table, schema[table], core_ontology)
        core_ontology = merge_ontologies_ttl(core_ontology, delta)
        print("✅")
        successful.append(table)
    except Exception as e:
        print(f"❌ {e}")
        failed.append(table)

print(f"\nSuccess: {len(successful)}, Failed: {len(failed)}")
if failed:
    print(f"Failed tables: {failed}")

is_valid, error = validate_turtle_string(core_ontology)
print(f"Final ontology valid: {is_valid}")
if not is_valid:
    print(f"Error: {error}")

Processing 16 tables: ['granter_consortiumpartnertype', 'granter_generalopportunityfile', 'granter_opportunity', 'granter_application', 'granter_applicationfile', 'granter_company', 'granter_companyfile', 'granter_companymemory', 'granter_consortium', 'granter_consortiumpartner', 'granter_consortiumpartnertype_consortium', 'granter_eligibilitycriteria', 'granter_generalopportunityfile_opportunities', 'granter_matchgroup', 'granter_opportunityfile', 'granter_timeline']
Processing granter_consortiumpartnertype... JudgeLLM Validation failed. Corrected Version:
(True, '')
✅
Processing granter_generalopportunityfile... JudgeLLM Validation failed. Corrected Version:
(True, '')
✅
Processing granter_opportunity... JudgeLLM Validation failed. Corrected Version:
(True, '')
✅
Processing granter_application... JudgeLLM Validation failed. Corrected Version:
(False, 'at line 6 of <>:\nBad syntax (Prefix ":" not bound) at ^ in:\n"...b\'ema#> .\\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\\n\\

In [45]:
# Retry failed tables
if failed:
    print(f"\n🔄 Retrying {len(failed)} failed tables...")
    
    retry_successful = []
    retry_failed = []
    
    for table in failed:
        print(f"Retrying {table}...", end=" ")
        
        try:
            delta = build_table_ontology(table, schema[table], core_ontology)
            core_ontology = merge_ontologies_ttl(core_ontology, delta)
            print("✅")
            retry_successful.append(table)
        except Exception as e:
            print(f"❌ {e}")
            retry_failed.append(table)
    
    # Update the lists
    successful.extend(retry_successful)
    failed = retry_failed
    
    print(f"\nRetry Results:")
    print(f"✅ Newly successful: {len(retry_successful)}")
    print(f"❌ Still failed: {len(retry_failed)}")


🔄 Retrying 2 failed tables...
Retrying granter_application... JudgeLLM Validation succeeded.
(True, '')
✅
Retrying granter_applicationfile... JudgeLLM Validation failed. Corrected Version:
(True, '')
✅

Retry Results:
✅ Newly successful: 2
❌ Still failed: 0


In [47]:
validate_turtle_string(core_ontology)

(True, '')

In [46]:
print(core_ontology)

@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/ontology#GrantPayment> a owl:Class ;
    prov:wasDerivedFrom <http://example.org/provenance/grant_payment> .

<http://example.org/ontology#OrganisationalCriterion> a owl:Class ;
    prov:wasDerivedFrom <http://example.org/provenance/OrganisationalCriterion> .

<http://example.org/ontology#admissibleProjects> a owl:DatatypeProperty ;
    rdfs:domain <http://example.org/ontology#granter_opportunity> ;
    rdfs:range xsd:string ;
    prov:wasDerivedFrom <http://example.org/provenance/granter_opportunity/admissible_projects> .

<http://example.org/ontology#aiReviewState> a owl:DatatypeProperty ;
    rdfs:domain <http://example.org/ontology#GranterApplication> ;
    rdfs:range xsd:string ;
    prov:wasDerivedFrom <http://example.org/provenance/granter_application/ai_review_

In [48]:
def save_and_validate_ttl(ontology_string: str, filename: str = None):
    """
    Save ontology string as .ttl file with validation
    """
    # Validate first
    is_valid, error = validate_turtle_string(ontology_string)
    if not is_valid:
        print(f"Invalid Turtle syntax: {error}")
        return None
    
    output_dir = "../../output/ontologies/RDB"
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    
    try:
        g = Graph()
        g.parse(data=ontology_string, format="turtle")
        g.serialize(destination=filepath, format="turtle")
        
        print(f"Ontology saved: {filename}")
        print(f"{len(g)} triples")
        print(f"{os.path.getsize(filepath):,} bytes")
        return filepath
    except Exception as e:
        print(f"Save failed: {e}")
        return None

save_and_validate_ttl(core_ontology, 'rigor_ontology.ttl')

Ontology saved: rigor_ontology.ttl
951 triples
66,568 bytes


'../../output/ontologies/RDB\\rigor_ontology.ttl'